In [19]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter

%config InlineBackend.figure_format='retina'

%matplotlib qt

In [20]:
class solarsys_body(object):
    def __init__(self, mass, radius_x, radius_y, velocity_x, velocity_y):
        self.mass = mass
        self.radius_x = radius_x
        self.radius_y = radius_y
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y
    def position(self):
        return np.array([self.radius_x, self.radius_y])
    def velocity(self):
        return np.array([self.velocity_x, self.velocity_y])
    def vector_SuntoEarth(self):       #Vector from Sun to the Earth
        return (self.radius_x,self.radius_y)

In [21]:
class star(object):  #This is for stars we are getting parallax for... not the SUN
    def __init__(self, mass_star, distance):
        self.mass_star = mass_star
        self.distance = distance       #distance from Earth
    def vector_SuntoStar(self):        #Vector from Sun to Star
        return (0, self.distance)

In [22]:
class vector_op(solarsys_body, star):  #This calculates the parallax angle seen from Earth
    def __init__(self, solarsys_body, star):
        self.vector_SuntoEarth = solarsys_body.vector_SuntoEarth()
        self.vector_SuntoStar = star.vector_SuntoStar()
        self.vector_EarthtoStarx = self.vector_SuntoStar[0] - self.vector_SuntoEarth[0]
        self.vector_EarthtoStary = self.vector_SuntoStar[1] - self.vector_SuntoEarth[1]
    def vector_EarthtoStar(self):
        return (self.vector_EarthtoStarx, self.vector_EarthtoStary)
    def dot_prod(self):
        return (self.vector_EarthtoStarx * self.vector_SuntoStar[0]) + (self.vector_EarthtoStary * self.vector_SuntoStar[1])
    def norm_prod(self):
        return (np.sqrt(self.vector_EarthtoStarx**2 + self.vector_EarthtoStary**2)) * (np.sqrt(self.vector_SuntoStar[0]**2 + self.vector_SuntoStar[1]**2))
    def parallax(self):
        angle_radian = np.arccos((self.dot_prod()) / (self.norm_prod()))
        angle_arcsec = angle_radian * ((3600*180)/np.pi)
        return angle_arcsec

In [23]:
#SOLAR SYSTEM = solarsys_body(mass, radius_x, radius_y, veloc_x, veloc_y)

earth = solarsys_body(3e-5, 1, 0, 0, 1)  #NOTE: mass is in solar masses and distance in AU
sun = solarsys_body(1, 0, 0, 0, 0)

#OUTER SOLAR SYSTEM STAR = star(mass_star, distance)

prox_cent = star(0.125, 267071.671)        # 0.77233 arcsec      
alpha_cenA = star(2, 277941.826)           # 0.74212 arcsec
barnard = star(0.16, 375711.345)           # 0.54901 arcsec
lalande = star(0.46, 525645.232)           # 0.39240 arcsec

#data from: http://csep10.phys.utk.edu/OJTA2dev/ojta/c2c/ordinary_stars/distances/nearby_tl.html

betelgeuse = star(11, 34673114)            # 5.95 miliarcsec  or 0.00595 arcsec


'''
STEP 1: ADDING STAR TO OUR DATA
name_of_star = star(star_mass, distance from earth in AU)
'''
        
#To find the parallax angle of an individual star:
EarthtoAlphaCenA = vector_op(earth, alpha_cenA)
EarthtoAlphaCenA.parallax()

0.7421164763979982

In [24]:
mp = earth.mass           #EARTH mass
ms = sun.mass             #SUN mass

r1 = earth.position()     #EARTH radius (position)        
r2 = sun.position()       #SUN radius (position)

v1 = earth.velocity()     #EARTH velocity     
v2 = sun.velocity()       #SUN velocity

In [25]:
def EarthOrbit(w, t, m1, m2):
    r1 = w[:2]
    v1 = w[2:4]
    
    r12 = np.linalg.norm(r1)
    
    dv1bydt = m2*(-r1)/r12**3
    dr1bydt = v1
    
    r_derivs = dr1bydt
    v_derivs = dv1bydt
    
    derivs = np.concatenate((r_derivs, v_derivs))
    
    return derivs

In [26]:
init_params = np.array([r1, v1])
init_params = init_params.flatten()
time_span = np.linspace(0, 6.2845, 365)

sol = scipy.integrate.odeint(EarthOrbit, init_params, time_span, args=(mp,ms))
                                                                       
r1_sol = sol[:,:2]

In [27]:
#Creating a list of parallax angles:
list_parallax = []

for i in range(len(time_span)):
    radiusx_iterated = r1_sol[i,0]
    radiusy_iterated = r1_sol[i,1]
    earth_iterated = solarsys_body(3e-5, radiusx_iterated, radiusy_iterated, 0, 1)
    
    EarthtoAlphaCenA_iterated = vector_op(earth_iterated, alpha_cenA)
    parallax_angles = EarthtoAlphaCenA_iterated.parallax()
    
    list_parallax.append(parallax_angles)
    
'''
STEP 2: CREATING AN ARRAY OF PARALLAX ANGLES
    EarthtoNameOfStar_iterated = vector_op(earth_iterated, name_of_star)
    parallax_angles = EarthtoNameOfStar_iterated.parallax()

'''
    
#Setting negative values for parallax depending on the position of the Earth while rotating:
first_quarter = (-1) * np.array([ list_parallax[0:91] ])
second_quarter = np.array([ list_parallax[91:182] ])
third_quarter = np.array ([ list_parallax[182:273] ])
fourth_quarter = (-1) * np.array ([ list_parallax[273:365] ])

#Flattening the lists and sorting the data into months:
January_March = [item for sublist in first_quarter for item in sublist]
April_June = [item for sublist in second_quarter for item in sublist]
July_September = [item for sublist in third_quarter for item in sublist]
October_December = [item for sublist in fourth_quarter for item in sublist]

#Creating one array with all values of parallax throughout the year:
all_year = np.array( January_March + April_June + July_September + October_December )


In [29]:
img1 = plt.imread("night_sky.jpg")

#Initializing the writer:
metadata = dict(title='Python Decal Final Project', artist='TEAM 11')
writer = FFMpegWriter(fps=10, metadata=metadata, bitrate=200000)
fig = plt.figure(dpi=200)

#making plot background black:
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = 'black' 

#Writer loop:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

with writer.saving(fig, "Parallax - Alpha Centauri.mp4", dpi=200):
    for i in range(len(time_span)):

        ax1.clear()
        ax2.clear()
        
#------------------------------------------------------MAIN PLOT-----------------------------------------------------        
        
        ax1.grid(False)
        ax1.axis("off")
    
        #Plot for orbit tail:
        ax1.plot(r1_sol[:i, 0], r1_sol[:i, 1], color = "lightcyan", alpha = 0.5)

        #Earth
        ax1.scatter(r1_sol[i,0], r1_sol[i,1], color = "cornflowerblue", marker = "o", s = 50, zorder = 5)
        
        #Sun
        ax1.scatter(0, 0, color="yellow",marker="o",s=175, zorder=5)
        
        #Distant Star
        ax1.scatter(0, 10, color="orange", marker="*", s=135, zorder=5)
        
        #Line from Sun to distant Star
        ax1.plot((0,0), (0,15),color="red", alpha=0.5)
        
        #Line from Earth to Star:
        ax1.plot((0,r1_sol[i,0]), (10,r1_sol[i,1]), color="blue", alpha=0.5)
                                          #DONT FORGET!!: (x_top, x_bottom), (y_top, y_bottom)
        
        #Text (Distance = -- AU):
        ax1.text(-0.1, 3, "Distance = 267071 AU", fontsize = 10, color="red", rotation=90)
        
        #Text (Name of Star):
        ax1.text(-1.25, 9.85, "Alpha Centauri A --->", fontsize = 12, color="orange")
        
        '''
        STEP 3: CHANGE THE NAME OF THE STAR ON THE ANIMATION:
        ax1.text(-1.30, 9.85, 'Name of Star --->', fontsize = 12, color='orange')
        '''
        
        #Text (Parallax Angle):
        radiusx_iterated = r1_sol[i,0]
        radiusy_iterated = r1_sol[i,1]
        earth_iterated = solarsys_body(3e-5, radiusx_iterated, radiusy_iterated, 0, 1)
        
        EarthtoAlphaCenA_iterated = vector_op(earth_iterated, alpha_cenA)
        EarthtoAlphaCenA_iterated.parallax()
        ax1.text(((r1_sol[i,0]/(-2))-0.5), 15.5, "Parallax Angle: \n    {:.10f}\" ".format(EarthtoAlphaCenA_iterated.parallax()), fontsize=12, color="white")

        '''
        STEP 4: SETTING THE VALUES OF THE PARALLAX ANGLE IN THE MAIN ANIMATION: 

        EarthtoNameOfStar_iterated = vector_op(earth_iterated, name_of_star)
        EarthtoNameOfStar_iterated.parallax()
        ax1.text(((r1_sol[i,0]/(-2))-0.5), 15.5, "Parallax Angle: \n    {:.5f}\" ".format(EarthtoNameOfStar_iterated.parallax()), fontsize=12, color="white")        
        '''
        
        #The apparent movement of the distant star:
        ax1.scatter((r1_sol[i,0]/(-2)), 15, color='white', marker = "*", s=135, zorder=5)
        
        #Dashed lines at maximum parallax:
        ax1.plot((-0.5, 1), (15, 0), color='#465fb0', linestyle='dashed')
        if (182 <= i):
            ax1.plot((0.5, -1), (15, 0), color='#465fb0', linestyle='dashed')
        
        #Dashed trail of the aparent movement of the distant star:
        ax1.plot((0, (r1_sol[i,0]/(-2))), (15, 15), color='white', linestyle='dashed', alpha=0.5)
        
        #Line from distant star to apparent star moving:
        ax1.plot((0, (r1_sol[i,0]/(-2))), (10, 15),  color="blue", alpha=0.5)

        
        ax1.set_xlim(-1.5, 1.5)
        ax1.set_ylim(-1.5, 17)

        
#----------------------------------------------------SUBPLOT------------------------------------------------------
        
        #Distant star in subplot:
        ax2.imshow(img1, extent=[-1.5, 1.5, -1.5, 1.5])
        
        ax2.grid(False)
        ax2.axis(False)
        
        #star
        ax2.scatter(all_year[i], 0.5, color="orange", marker="*",s=135, zorder=5)
        
        #dashed line moving with star
        ax2.plot((0, all_year[i]), (0.5, 0.5), color="#bcc3cc", linestyle="dashed")
        
        #Month label
        if (i <= 31) or (i == 364):
            ax2.text(-1.25, 1.25, "January", fontsize=20, color="#bcc3cc")
        if (31 < i <= 59):
            ax2.text(-1.25, 1.25, "February", fontsize=20, color="#bcc3cc")
        if (59 < i <= 90):
            ax2.text(-1.25, 1.25, "March", fontsize=20, color="#bcc3cc")
        if (90 < i <= 120):
            ax2.text(-1.25, 1.25, "April", fontsize=20, color="#bcc3cc")
        if (120 < i <= 151):
            ax2.text(-1.25, 1.25, "May", fontsize=20, color="#bcc3cc")
        if (151 < i <= 181):
            ax2.text(-1.25, 1.25, "June", fontsize=20, color="#bcc3cc")
        if (181 < i <= 212):
            ax2.text(-1.25, 1.25, "July", fontsize=20, color="#bcc3cc")
        if (212 < i <= 243):
            ax2.text(-1.25, 1.25, "August", fontsize=20, color="#bcc3cc")
        if (243 < i <= 273):
            ax2.text(-1.25, 1.25, "September", fontsize=20, color="#bcc3cc")
        if (273 < i <= 304):
            ax2.text(-1.25, 1.25, "October", fontsize=20, color="#bcc3cc")
        if (304 < i <= 334):
            ax2.text(-1.25, 1.25, "November", fontsize=20, color="#bcc3cc")
        if (334 < i < 363):
            ax2.text(-1.25, 1.25, "December", fontsize=20, color="#bcc3cc")
            
        #Parallax displayed
        if (i <= 91) or (273 < i <= 365):
            ax2.text(-0.55, 0.35, "{:.5f}\" ".format(EarthtoAlphaCenA_iterated.parallax()), fontsize=10, color="white")
        if (91 < i <= 273):
            ax2.text(0, 0.35, "{:.5f}\" ".format(EarthtoAlphaCenA_iterated.parallax()), fontsize=12, color="white")

        '''
        STEP 5: CHANGE THE PARALLAX ANGLE DISPLAYED IN THE SUBPLOT:
        if (i <= 91) or (273 < i <= 365):
            ax2.text(-0.55, 0.35, "{:.5f}\" ".format(EarthtoNameOfStar_iterated.parallax()), fontsize=10, color="white")
        if (91 < i <= 273):
            ax2.text(0, 0.35, "{:.5f}\" ".format(EarthtoNameOfStar_iterated.parallax()), fontsize=12, color="white")
        '''
     
        ax2.set_xlim(-1.5, 1.5)
        ax2.set_ylim(-1.5, 1.5)
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '2400x1474', '-pix_fmt', 'rgba', '-r', '10', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-b', '200000k', '-metadata', 'title=Python Decal Final Project', '-metadata', 'artist=TEAM 11', '-y', 'Parallax - Alpha Centauri.mp4']' returned non-zero exit status 255.